The following code defines hyperparameter tuning for LSTM model designed for uni-modal dataset. 

In [1]:
import pandas as pd
import os
import numpy as np
import json

In [2]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

from sklearn.preprocessing import StandardScaler

2022-05-21 15:47:04.413415: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-21 15:47:04.413452: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import sys
sys.path.insert(1, '../')

from data_preparation import prepare_x_data, get_Y_labels, reshape_Y, reshape_X

In [4]:
scaler = StandardScaler()

## Data preparation

### X data - audio and visual features

In [5]:
x_train_visual = prepare_x_data('../Data/LLDs_video_openface/train',',', 5, scaler)
x_train_audio = prepare_x_data('../Data/LLDs_audio_eGeMAPS/train',';', 2, scaler) 

### Y data - YMRS score

In [6]:
y_train = get_Y_labels('../Data/labels_metadata.csv', 60, 164, scaler)
y_train = reshape_Y(y_train,len(x_train_visual),1,1)

## Hyperparameter Tuner setup

In [7]:
def model_builder_visual(hp):
    
    model = keras.Sequential()
    
    # Hyperparameters for tuning
    hp_dense_units_1 = hp.Int('units_1', min_value=1, max_value=465, step=1)
    hp_dense_units_2 = hp.Int('units_2', min_value=1, max_value=465, step=1)
    hp_droput = hp.Choice('dropout', values=[1e-2, 1e-3, 1e-4])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]) #learning rate for the optimizer
    
    model.add(layers.LSTM(
                        units = hp_dense_units_1, 
                        input_shape=(None,465), 
                        return_sequences=True 
                    ))
    model.add(layers.Dropout(hp_droput))
    model.add(layers.LSTM(
                        units = hp_dense_units_2, 
                        input_shape=(None,hp_dense_units_1), 
                        return_sequences=False 
                    ))
    model.add(layers.Dropout(hp_droput))
    model.add(layers.Dense(1, activation='linear'))

    model.compile(loss='mse', 
                  metrics=[keras.metrics.MeanAbsoluteError()] ,
                  optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  ) 
    
    return model

In [8]:
def model_builder_audio(hp):
    model = keras.Sequential()
    
    hp_dense_units_1 = hp.Int('units_1', min_value=1, max_value=23, step=1)
    hp_dense_units_2 = hp.Int('units_2', min_value=1, max_value=23, step=1)
    hp_droput = hp.Choice('dropout', values=[1e-2, 1e-3, 1e-4])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]) #learning rate for the optimizer
    
    model.add(layers.LSTM(
                        units = hp_dense_units_1, 
                        input_shape=(None,23), 
                        return_sequences=True 
                    ))
    model.add(layers.Dropout(hp_droput))
    model.add(layers.LSTM(
                        units = hp_dense_units_2, 
                        input_shape=(None,hp_dense_units_1), 
                        return_sequences=False 
                    ))
    model.add(layers.Dropout(hp_droput))
    model.add(layers.Dense(1, activation='linear'))

    model.compile(loss='mse', 
                  metrics=[keras.metrics.MeanAbsoluteError()] ,
                  optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  ) 
    
    return model

In [9]:
#Initilize tuners - for video and audio modality seperately

tuner_visual = kt.Hyperband(
    model_builder_visual,
    objective = 'loss',
    max_epochs = 50,
    factor = 3,
    project_name = 'Hp_tuner_visual'
)

tuner_audio = kt.Hyperband(
    model_builder_audio,
    objective = 'loss',
    max_epochs = 50, 
    factor = 3,
    project_name = 'Hp_tuner_audio'
)

INFO:tensorflow:Reloading Oracle from existing project ./Hp_tuner_visual/oracle.json


2022-05-21 15:47:44.052987: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-05-21 15:47:44.058092: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-21 15:47:44.058164: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-05-21 15:47:44.058199: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-05-21 15:47:44.058233: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

INFO:tensorflow:Reloading Tuner from ./Hp_tuner_visual/tuner0.json
INFO:tensorflow:Reloading Oracle from existing project ./Hp_tuner_audio/oracle.json
INFO:tensorflow:Reloading Tuner from ./Hp_tuner_audio/tuner0.json


In [10]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

In [11]:
# Train audio data tuner
index = 0
for file in x_train_audio:
    file = reshape_X(file)
    tuner_audio.search(
        file, 
        y_train[index], 
        epochs=10,  
        batch_size = len(file), 
        callbacks=[stop_early]) 

INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorf

In [12]:
#Train visual data tuner
index = 0
for file in x_train_visual:
    file = reshape_X(file)
    tuner_visual.search(
        file, 
        y_train[index], 
        epochs=10, 
        batch_size = len(file),
        callbacks=[stop_early])

Trial 128 Complete [00h 02m 22s]
loss: 0.015372773632407188

Best loss So Far: 6.27892973170674e-08
Total elapsed time: 00h 18m 59s
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered

### Finding the final hyperparameter values

The approach of finding the final hyperparameters for the model is based on extracting the average value from all tuner search iterations for each modality seperately. 

In [13]:
# Combining learned hyperparameters from every iteration in one dataset

def get_hp_df(tunerdir):
    rootdir = tunerdir
    hp_values = []
    #Get learned hyperparameters for every file (trial) and append to a list
    for trial in os.listdir(rootdir):
        pathdir = os.path.join(rootdir, trial)
        filedir = os.path.join(pathdir, 'trial.json')
        
        if os.path.isdir(pathdir): #looking only for trial subfolders
            with open(filedir) as json_file:
                data = json.load(json_file)
                values = data['hyperparameters']['values'] #get learned hyperparameters
                hp_values.append(values) #append to list
        
    #Transform to dataframe
    hp_df = pd.DataFrame(hp_values)

    return hp_df

In [16]:
#Get hyperparameter values for both modalities
hpdf_visual = get_hp_df('../LSTM/Hp_tuner_visual')
hpdf_audio = get_hp_df('../LSTM/Hp_tuner_audio')

In [17]:
print(""" Hyperparameters for the LSTM model for each data modality.
--------FOR VISUAL DATA:""")
print(hpdf_visual.mean())

print("--------FOR AUDIO DATA:")
print(hpdf_audio.mean())

 Hyperparameters for the LSTM model for each data modality.
--------FOR VISUAL DATA:
units_1                206.203125
units_2                207.382812
dropout                  0.003672
learning_rate            0.002533
tuner/epochs            15.078125
tuner/initial_epoch      2.437500
tuner/bracket            2.078125
tuner/round              0.390625
dtype: float64
--------FOR AUDIO DATA:
units_1                12.148438
units_2                11.460938
dropout                 0.003925
learning_rate           0.003707
tuner/epochs           15.078125
tuner/initial_epoch     2.437500
tuner/bracket           2.078125
tuner/round             0.390625
dtype: float64


/tmp/ipykernel_3205/2966352655.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(hpdf_visual.mean())
/tmp/ipykernel_3205/2966352655.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(hpdf_audio.mean())
